In [ ]:
import sys

sys.path.append("../src")
from utils.metrics_hai import compute_metrics
from utils.utils import loss_01

logging.getLogger().setLevel(logging.INFO)
import math

# import os, json, cv2, random
# import some common detectron2 utilities
import os
import pickle

import matplotlib.pyplot as plt

# Some basic setup:
# Setup detectron2 logger
# import some common libraries
import numpy as np
import pandas as pd
import requests
import torch
import torchvision.datasets as dset
import torchvision.transforms as transforms
from PIL import Image
from sklearn.metrics import classification_report
from tqdm import tqdm
from transformers import CLIPModel, CLIPProcessor

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
print(torch.cuda.is_available())
from datasets_hai.coco import *
from describers.domino_describe import *
from describers.itterative_describe import *
from describers.seal_describe import *
from utils.metrics_caption import *
from utils.metrics_hai import *
from utils.metrics_regions import *
from utils.utils import *

In [ ]:
# OPEN AI KEY
keyfile = open("../keys.txt", "r")
# read the file
key = keyfile.read()


ms-coco dataset prep

In [ ]:
validation_coco_captions = dset.CocoCaptions(root ="../data/coco/val2017",
                        annFile ="../data/coco/annotations/captions_val2017.json",
                        transform=transforms.Compose([transforms.PILToTensor(),pil_to_detectronformat]))
captions = []
for i in tqdm(range(len(validation_coco_captions))):
    captions.append(validation_coco_captions[i][1][0])
captions_clean = []
for caption in captions:
    caption = caption.replace("\n", " ").replace("\r", " ").replace('"', '')
    captions_clean.append(caption)
captions = np.array(captions_clean)
data_coco_storred = pickle.load(open("../data" + "/coco/data_coco_embs_preds.pkl","rb"))
label_to_category_rcnn = data_coco_storred["label_to_category_rcnn"]

dataset = pickle.load(open("../data/cleaned_pkl/coco_dataset.pkl","rb"))

metadata = [['' for _ in range(len(dataset.metadata[0]))] for __ in range(len(dataset.metadata))]
for i in range(len(dataset.metadata)):
    for j in range(len(dataset.metadata[0])):
        if 'present' in dataset.metadata[i][j]:
            metadata[i][j] =  "present"
        else:
            metadata[i][j] =  "absent"

dataset.metadata = metadata

In [ ]:

pre_instruction = "I will provide you with a set of descriptions of points that belong to a region and a set of descriptions of point that do not belong to the region." + \
    "Your task is to summarize the points inside the region in a concise and precise short sentence while making sure the summary contrast to points outside the region." + \
    "Your one sentence summary should be able to allow a person to distinguish between points inside and outside the region while describing the region well." + \
    "The summary should be a single word, it should be accurate, concise, distinguishing and precise." + \
    "Example: \n" + "inside the region: \n two cows and two sheep grazing in a pasture. \n the sheep is standing near a tree. \n not in the region:  the cows are lying on the grass beside the water.\n" + \
    "summary: sheep. \n End of Example \n"

pre_instruction_no_contrast = "I will provide you with a set of descriptions of points that belong to a region." + \
    "Your task is to summarize the points inside the region in a concise and precise short sentence ." + \
    "Your one sentence summary should be able to allow a person to distinguish  points inside the region while describing the region well." + \
    "The summary should be a single word, it should be accurate, concise, distinguishing and precise." + \
    "Example: \n" + "inside the region: \n two cows and two sheep grazing in a pasture. \n the sheep is standing near a tree.\n" + \
    "summary: sheep. \n End of Example \n"


In [ ]:
unique_dim = []
metrics_seal = []
metrics_domino = []
metrics_itt5 = []
metrics_itt10 = []
metrics_itt_nocontrast = []
metrics_itt_1round = []
metrics_itt_chat = []

for i in range(73): # 73 is the limit
    while True:
        region_dims = random.sample(list(range(len(dataset.metadata[0]))),1)
        if label_to_category_rcnn[region_dims[0]] in unique_dim:
            continue
        region_conds = ['present']#, 'absent']
        cluster_scores = np.zeros((len(dataset.data_y), 2))
        cluster_labels = np.zeros(len(dataset.data_y))
        in_region_count = 0
        index_inside = 0

        for i in range(len(dataset.data_y)):
            # check metadata condition

            in_region = all([dataset.metadata[i][region_dims[j]] == region_conds[j] for j in range(len(region_dims))])
            if in_region:
                in_region_count += 1
                index_inside = i
                cluster_scores[i] = [0,1]
                cluster_labels[i] = 1
            else:
                cluster_scores[i] = [1,0]
                cluster_labels[i] = 0
        in_region_present = 0 
        for i in range(len(dataset.data_y)):
            # check metadata condition
            in_region = all([dataset.metadata[i][region_dims[j]] == region_conds[j] for j in range(1)])
            if in_region:
                in_region_present += 1
                cluster_scores[i] = [0,1]
                cluster_labels[i] = 1
            else:
                cluster_scores[i] = [1,0]
                cluster_labels[i] = 0

        if in_region_count >= 50:
            #print(in_region_present)
            #print(in_region_count)
            embedding_inside = dataset.data_x[index_inside]
            cosine_distances = pairwise_distances(embedding_inside.reshape(1, -1), dataset.data_x, metric='cosine')
            cosine_distances = cosine_distances.reshape(-1)
            # update cluter scores
            #cluster_scores[:,0] = cosine_distances
            #cluster_scores[:,1] = 1 - cosine_distances

            break
    cluster_labels = cluster_labels.astype(int)
    unique_dim.append(label_to_category_rcnn[region_dims[0]])
    print(len(unique_dim))
    ground_truth1 = [label_to_category_rcnn[region_dims[0]]]# +" " + region_conds[0]] #+ " and " +  label_to_category_rcnn[region_dims[1]] + " " +  region_conds[1]]
    print(ground_truth1)


    domino_desriber = DOMINODescribe(captions, dataset.data_x, cluster_labels)
    domino_des = domino_desriber.describe_region(1)
    
    seal_describer = SEALDescribe(captions, cluster_labels, 'object detection', key)
    seal_des = seal_describer.describe_region(1)
    
    domino_metrics =  evaluate_captions(ground_truth1, [domino_des])
    seal_metrics = evaluate_captions(ground_truth1, [seal_des[0]])
    metrics_seal.append(seal_metrics)
    metrics_domino.append(domino_metrics)
    
    # ORIGINAL VERSION 5
    itt_desriber = IterativeRegionDescribe(captions, dataset.data_x, cluster_scores, cluster_labels, key, get_text_embeddings,
                                           5,  initial_positive_set_size = 15, initial_negative_set_size = 5)
    itt_desriber.pre_instruction = pre_instruction
    itt_des = itt_desriber.describe_region(1)
    itt_metrics = evaluate_captions(ground_truth1, [itt_des[0][-1]])
    metrics_itt5.append(itt_metrics)
    
    # ORIGINAL VERSION 10 
    itt_desriber = IterativeRegionDescribe(captions, dataset.data_x, cluster_scores, cluster_labels, key, get_text_embeddings,
                                           10,  initial_positive_set_size = 15, initial_negative_set_size = 5)
    itt_desriber.pre_instruction = pre_instruction
    itt_des = itt_desriber.describe_region(1)
    itt_metrics = evaluate_captions(ground_truth1, [itt_des[0][-1]])
    metrics_itt10.append(itt_metrics)

    # ONE SHOT NO CONTRASTING
    itt_desriber = IterativeRegionDescribe(captions, dataset.data_x, cluster_scores, cluster_labels, key, get_text_embeddings,
                                           0,  initial_positive_set_size = 20, initial_negative_set_size = 0)
    itt_desriber.pre_instruction = pre_instruction_no_contrast
    itt_des = itt_desriber.describe_region(1)
    itt_metrics = evaluate_captions(ground_truth1, [itt_des[0][-1]])
    metrics_itt_nocontrast.append(itt_metrics)
    
    # ONE SHOT  CONTRASTING
    itt_desriber = IterativeRegionDescribe(captions, dataset.data_x, cluster_scores, cluster_labels, key, get_text_embeddings,
                                           0,  initial_positive_set_size = 20, initial_negative_set_size = 10)
    itt_desriber.pre_instruction = pre_instruction
    itt_des = itt_desriber.describe_region(1)
    itt_metrics = evaluate_captions(ground_truth1, [itt_des[0][-1]])
    metrics_itt_1round.append(itt_metrics)
    
    # ORIGINAL VERSION CHAT
    itt_desriber = IterativeRegionDescribe(captions, dataset.data_x, cluster_scores, cluster_labels, key, get_text_embeddings,
                                           5,  initial_positive_set_size = 15, initial_negative_set_size = 5, chat_correct = True)
    itt_desriber.pre_instruction = pre_instruction
    itt_des = itt_desriber.describe_region(1)
    itt_metrics = evaluate_captions(ground_truth1, [itt_des[0][-1]])
    metrics_itt_chat.append(itt_metrics)
    metrics_to_print = ['METEOR', 'sent-sim','ROUGE','SPICE']

    for metric_print in  metrics_to_print:
        print(f' domino {metric_print} {np.mean([x[metric_print] for x in metrics_domino])} {np.std([x[metric_print] for x in metrics_domino])/np.sqrt(len(metrics_seal))}')
        print(f' seal {metric_print} {np.mean([x[metric_print] for x in metrics_seal])} {np.std([x[metric_print] for x in metrics_seal] )/np.sqrt(len(metrics_seal))}')
        print(f' itt5 {metric_print} {np.mean([x[metric_print] for x in metrics_itt5])} {np.std([x[metric_print] for x in metrics_itt5])/np.sqrt(len(metrics_seal))}')
        print(f' itt10 {metric_print} {np.mean([x[metric_print] for x in metrics_itt10])} {np.std([x[metric_print] for x in metrics_itt10])/np.sqrt(len(metrics_seal))}')
        print(f' itt no contrast {metric_print} {np.mean([x[metric_print] for x in metrics_itt_nocontrast])} {np.std([x[metric_print] for x in metrics_itt_nocontrast])/np.sqrt(len(metrics_seal))}')
        print(f' itt 1 round {metric_print} {np.mean([x[metric_print] for x in metrics_itt_1round])} {np.std([x[metric_print] for x in metrics_itt_1round])/np.sqrt(len(metrics_seal))}')
        print(f' itt chat {metric_print} {np.mean([x[metric_print] for x in metrics_itt_chat])} {np.std([x[metric_print] for x in metrics_itt_chat])/np.sqrt(len(metrics_seal))}')
        print('\n')
    

In [ ]:
    for metric_print in  metrics_to_print:
        print(f' domino {metric_print} {np.mean([x[metric_print] for x in metrics_domino])} {np.std([x[metric_print] for x in metrics_domino])/np.sqrt(len(metrics_seal))}')
        print(f' seal {metric_print} {np.mean([x[metric_print] for x in metrics_seal])} {np.std([x[metric_print] for x in metrics_seal] )/np.sqrt(len(metrics_seal))}')
        print(f' itt5 {metric_print} {np.mean([x[metric_print] for x in metrics_itt5])} {np.std([x[metric_print] for x in metrics_itt5])/np.sqrt(len(metrics_seal))}')
        print(f' itt10 {metric_print} {np.mean([x[metric_print] for x in metrics_itt10])} {np.std([x[metric_print] for x in metrics_itt10])/np.sqrt(len(metrics_seal))}')
        print(f' itt no contrast {metric_print} {np.mean([x[metric_print] for x in metrics_itt_nocontrast])} {np.std([x[metric_print] for x in metrics_itt_nocontrast])/np.sqrt(len(metrics_seal))}')
        print(f' itt 1 round {metric_print} {np.mean([x[metric_print] for x in metrics_itt_1round])} {np.std([x[metric_print] for x in metrics_itt_1round])/np.sqrt(len(metrics_seal))}')
        print(f' itt chat {metric_print} {np.mean([x[metric_print] for x in metrics_itt_chat])} {np.std([x[metric_print] for x in metrics_itt_chat])/np.sqrt(len(metrics_seal))}')
        print('\n')